In [ ]:
from IPython.display import display, clear_output
import ipywidgets as ipw
import widgets
import utils
import os
import json
import shutil

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

product_concept_selector = widgets.ObjectSelectionWidget("Reaction Product Concept")
product_concept_selector.load_dropdown_box()

chemist_selector = widgets.ObjectSelectionWidget("Chemist")
chemist_selector.load_dropdown_box()

storage_selector = widgets.ObjectSelectionWidget("Storage")
storage_selector.load_dropdown_box()

supplier_selector = widgets.ObjectSelectionWidget("Instrument")
supplier_selector.load_dropdown_box()

product_widgets = widgets.ObjectPropertiesWidgets("ReactionProduct")
product_widgets.get_properties_widgets()

product_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def create_product_openbis(b):
        
    object_properties = utils.get_widget_values(product_widgets.properties_widgets_detailed_dict)

    # Check if the product is already in openBIS
    products_openbis = [f"{product.props['$name']}" for product in utils.get_openbis_objects(OPENBIS_SESSION, type ="REACTION_PRODUCT_CONCEPT")]
    
    product_name = object_properties['$name']
    if product_name in products_openbis:
        display(utils.Javascript(data = "alert('Reaction product concept is already in openBIS!')"))
        return

    object_parents = []
    
    if product_concept_selector.dropdown.value != -1:
        object_parents.append(product_concept_selector.dropdown.value)
        
    if chemist_selector.dropdown.value != -1:
        object_parents.append(chemist_selector.dropdown.value)
        
    if supplier_selector.dropdown.value != -1:
        object_parents.append(supplier_selector.dropdown.value)
    
    if storage_selector.dropdown.value != -1:
        object_parents.append(storage_selector.dropdown.value)

    product_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="REACTION_PRODUCT_CONCEPT",
        collection="/MATERIALS/MOLECULES/PRODUCT_COLLECTION", 
        props = object_properties,
        parents = object_parents
    )
    
    utils.upload_datasets(OPENBIS_SESSION, product_object, product_support_files, "RAW_DATA")
    display(utils.Javascript(data = "alert('Upload successful!')"))
    product_widgets.reset_properties_widgets()

# Create reaction product

## Select reaction product concept

In [ ]:
display(product_concept_selector)

## Select chemist

In [ ]:
display(chemist_selector)

## Select supplier

In [ ]:
display(supplier_selector)

## Select storage

In [ ]:
display(storage_selector)

## Properties

In [ ]:
display(product_widgets)

## Support files

In [ ]:
display(product_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_product_openbis)
quit_button.on_click(close_notebook)